# CHESS DATA ANALYSIS

TEAM: Abraham Borg, Mehar Rekhi, Sarom Thin, Cristian Vazquez

INTRO: We want to analyze a data set of chess games and use the information to make predictions about who will win a game of chess. The data is taken from this source: https://www.kaggle.com/datasets/milesh1/35-million-chess-games?resource=download 

DESCRIPTION OF SOURCE DATA SET: The data shows all game moves for White and Black. The date of each game is specified. We are also given the results of each game. We know how many moves each player made. Finally, we know what the ELO rating of each player was at the time of the game. There is a lot of information that we are not interested in so that will need to be cleaned up. Also, the list of moves is a long string, but we want to have each move occupy its own dataframe column.

The complete dataset is 3.5 million games (filesize about 2GB), but for this experiment we initially selected the first 30,000 games for analysis. It was clear that this sample was heavily skewed for White. So instead we opted to randomly select games.

The data set was loaded into the dataframe from the local machine, and then the pickle of the completely prepared dataset was loaded onto github.

PREDICTION AND FEATURES TO USE AS PREDICTORS: 
PREDICTORS: The W-ELO, B-ELO, and Num Moves

PREDICT: Game Result, will White win based on the the predictors in our model?

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
from matplotlib import rcParams
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# allow output to span multiple output lines in the console
pd.set_option('display.max_columns', 500)

# switch to seaborn default stylistic parameters
# see the useful https://seaborn.pydata.org/tutorial/aesthetics.html

sns.set()
sns.set_context('paper') # 'talk' for slightly larger

# change default plot size
rcParams['figure.figsize'] = 9,7

# DATA PREPARATION

The data set is given with many columns that we don't need, data in the wrong format, and many records that are not useful. The data file is also very big. To make things easier we upload the data from the local machine and then generate a PKL file of the prepared dataframe.

In [2]:
filename = 'C:\\Users\\Abrah\\Desktop\\CST383 - Data Science\\Final Project\\Chess_Data.txt'
n = 3561470     #number of records in file
s = 450000      #desired sample size
skip = sorted(random.sample(range(n), n - s))

In [3]:
# column names, without the chess moves column
misc_columnNames = ['PNG_File_Pos - DELETE ME', 'Date', 'Game Result', 'W-ELO', 'B-ELO', 
                    'Num Moves', 'miscDate - DELETE ME', 'result - DELETE ME', 'wELO - DELETE ME', 'bELO - DELETE ME', 
                    'event date - DELETE ME', 'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 
                    'bad len - DELETE ME']
delete_cols = ['PNG_File_Pos - DELETE ME', 'miscDate - DELETE ME', 'result - DELETE ME', 
               'wELO - DELETE ME', 'bELO - DELETE ME', 'event date - DELETE ME', 
               'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 'bad len - DELETE ME']

In [4]:
# read all data except chess moves,
misc_chess_data = pd.read_csv(filename, comment = '#', infer_datetime_format = True, header = None, sep = ' ', on_bad_lines = 'skip', skiprows = skip)
misc_chess_data.drop(misc_chess_data.columns[16], axis = 1, inplace = True)
misc_chess_data.columns = misc_columnNames
misc_chess_data.drop(labels = ['PNG_File_Pos - DELETE ME', 'miscDate - DELETE ME', 'result - DELETE ME', 
               'wELO - DELETE ME', 'bELO - DELETE ME', 'event date - DELETE ME', 
               'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 'bad len - DELETE ME'], axis = 1, inplace = True)


C:\Users\Abrah\AppData\Local\Temp\ipykernel_28696\3410824919.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  misc_chess_data = pd.read_csv(filename, comment = '#', infer_datetime_format = True, header = None, sep = ' ', on_bad_lines = 'skip', skiprows = skip)


In [5]:
# sanity check
misc_chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves
0,2000.02.20,1/2-1/2,2851,2633,97
1,2000.02.12,1-0,2851,2555,147
2,1999.11.20,1-0,2851,None,41
3,1999.11.20,1-0,2851,None,55
4,2000.03.14,1-0,2851,None,61


In the above cell we can see that the B-ELO has None values. We will fix that shortly. Next we prepare isolate the chess moves from the other data to make it easier to prepare this portion of the data. The dataframe game_moves will be merged with the previous dataframe after processsing.

In [6]:
# Isolate game moves from everything else.
game_moves = pd.read_csv(filename, engine = 'python', sep = '###', on_bad_lines = 'skip', header = None, skiprows = skip)

# drop first column of game moves (this is the misc chess data)
game_moves.drop(game_moves.columns[0], axis = 1, inplace = True)

In [7]:
game_moves.head()

,1
0,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bb5 B3.a6 W4.Ba4...
1,W1.d4 B1.d5 W2.c4 B2.e6 W3.Nc3 B3.Nf6 W4.cxd5...
2,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bc4 B3.Bc5 W4.c3...
3,W1.d4 B1.d5 W2.c4 B2.dxc4 W3.e3 B3.Nf6 W4.Bxc...
4,W1.d4 B1.f5 W2.Bg5 B2.g6 W3.Nd2 B3.Bg7 W4.e4 ...


In [8]:
# split game moves df into columns for each move. 
game_moves = game_moves.iloc[:, 0].str.lstrip()
game_moves = game_moves.iloc[:].str.split(pat = ' ', expand = True)
game_moves.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359
0,W1.e4,B1.e5,W2.Nf3,B2.Nc6,W3.Bb5,B3.a6,W4.Ba4,B4.Nf6,W5.O-O,B5.Be7,W6.Re1,B6.b5,W7.Bb3,B7.d6,W8.c3,B8.O-O,W9.h3,B9.Na5,W10.Bc2,B10.c5,W11.d4,B11.Qc7,W12.Nbd2,B12.Bd7,W13.Nf1,B13.cxd4,W14.cxd4,B14.Rac8,W15.Ne3,B15.Nc6,W16.d5,B16.Nb4,W17.Bb1,B17.a5,W18.a3,B18.Na6,W19.b4,B19.Ra8,W20.Bd2,B20.Rfc8,W21.Bd3,B21.Qb7,W22.g4,B22.g6,W23.Nf1,B23.axb4,W24.axb4,B24.Bd8,W25.Ng3,B25.Nc7,W26.Qe2,B26.Rxa1,W27.Rxa1,B27.Ra8,W28.Qe1,B28.Nfe8,W29.Qc1,B29.Ng7,W30.Rxa8,B30.Qxa8,W31.Bh6,B31.Nce8,W32.Qb2,B32.Qa4,W33.Kg2,B33.Bb6,W34.Bc2,B34.Qa7,W35.Bd3,B35.Qa4,W36.Ne2,B36.Nc7,W37.Nxe5,B37.dxe5,W38.Qxe5,B38.Nce8,W39.Bxg7,B39.Qd1,W40.Bh6,B40.Qxd3,W41.Qe7,B41.Ng7,W42.Ng3,B42.Qc2,W43.Qf6,B43.Nf5,W44.Qxb6,B44.Nh4+,W45.Kh2,B45.Nf3+,W46.Kg2,B46.Nh4+,W47.Kh2,B47.Nf3+,W48.Kg2,B48.Nh4+,W49.Kh2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,W1.d4,B1.d5,W2.c4,B2.e6,W3.Nc3,B3.Nf6,W4.cxd5,B4.exd5,W5.Bg5,B5.c6,W6.Qc2,B6.Be7,W7.e3,B7.Nbd7,W8.Bd3,B8.O-O,W9.Nge2,B9.Re8,W10.O-O,B10.Nf8,W11.f3,B11.Ng6,W12.Rad1,B12.h6,W13.Bxf6,B13.Bxf6,W14.Bxg6,B14.fxg6,W15.e4,B15.g5,W16.e5,B16.Be7,W17.f4,B17.gxf4,W18.Nxf4,B18.Rf8,W19.Ng6,B19.Rxf1+,W20.Rxf1,B20.Be6,W21.Ne2,B21.Qd7,W22.h4,B22.Re8,W23.Ng3,B23.Bf7,W24.Nxe7+,B24.Rxe7,W25.Nf5,B25.Re6,W26.Nd6,B26.Bg6,W27.Qc3,B27.Rxd6,W28.exd6,B28.Qxd6,W29.Qa3,B29.Qb8,W30.Qe7,B30.Qg3,W31.Rf3,B31.Qg4,W32.Qf8+,B32.Kh7,W33.Qf4,B33.Qd7,W34.Rg3,B34.Be4,W35.Qe5,B35.Qf7,W36.h5,B36.Qd7,W37.b4,B37.a6,W38

In [9]:
# chess moves need to be strings, so convert objects to strings.
game_moves = game_moves.astype('string', copy=True, errors='raise')
game_moves.dtypes

0      string
1      string
2      string
3      string
4      string
        ...  
355    string
356    string
357    string
358    string
359    string
Length: 360, dtype: object

In [10]:
# merge misc data and game moves into one df.
chess_data = pd.concat([misc_chess_data, game_moves], axis = 1)

In [11]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359
0,2000.02.20,1/2-1/2,2851,2633,97,W1.e4,B1.e5,W2.Nf3,B2.Nc6,W3.Bb5,B3.a6,W4.Ba4,B4.Nf6,W5.O-O,B5.Be7,W6.Re1,B6.b5,W7.Bb3,B7.d6,W8.c3,B8.O-O,W9.h3,B9.Na5,W10.Bc2,B10.c5,W11.d4,B11.Qc7,W12.Nbd2,B12.Bd7,W13.Nf1,B13.cxd4,W14.cxd4,B14.Rac8,W15.Ne3,B15.Nc6,W16.d5,B16.Nb4,W17.Bb1,B17.a5,W18.a3,B18.Na6,W19.b4,B19.Ra8,W20.Bd2,B20.Rfc8,W21.Bd3,B21.Qb7,W22.g4,B22.g6,W23.Nf1,B23.axb4,W24.axb4,B24.Bd8,W25.Ng3,B25.Nc7,W26.Qe2,B26.Rxa1,W27.Rxa1,B27.Ra8,W28.Qe1,B28.Nfe8,W29.Qc1,B29.Ng7,W30.Rxa8,B30.Qxa8,W31.Bh6,B31.Nce8,W32.Qb2,B32.Qa4,W33.Kg2,B33.Bb6,W34.Bc2,B34.Qa7,W35.Bd3,B35.Qa4,W36.Ne2,B36.Nc7,W37.Nxe5,B37.dxe5,W38.Qxe5,B38.Nce8,W39.Bxg7,B39.Qd1,W40.Bh6,B40.Qxd3,W41.Qe7,B41.Ng7,W42.Ng3,B42.Qc2,W43.Qf6,B43.Nf5,W44.Qxb6,B44.Nh4+,W45.Kh2,B45.Nf3+,W46.Kg2,B46.Nh4+,W47.Kh2,B47.Nf3+,W48.Kg2,B48.Nh4+,W49.Kh2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000.02.12,1-0,2851,2555,147,W1.d4,B1.d5,W2.c4,B2.e6,W3.Nc3,B3.Nf6,W4.cxd5,B4.exd5,W5.Bg5,B5.c6,W6.Qc2,B6.Be7,W7.e3,B7.Nbd7,W8.Bd3,B8.O-O,W9.Nge2,B9.Re8,W10.O-O,B10.Nf8,W11.f3,B11.Ng6,W12.Rad1,B12.h6,W13.Bxf6,B13.Bxf6,W14.Bxg6,B14.fxg6,W15.e4,B15.g5,W16.e5,B16.Be7,W17.f4,B17.gxf4,W18.Nxf4,B18.Rf8,W19.Ng6,B19.Rxf1+,W20.Rxf1,B20.Be6,W21.Ne2,B21.Qd7,W22.h4,B22.Re8,W23.Ng3,B23.Bf7,W24.Nxe7+,B24.Rxe7,W25.Nf5,B25.Re6,W26.Nd6,B26.Bg6,W27.Qc3,B27.Rxd6,W28.exd6,B28.Qxd6,W29.Qa3,B29.Qb8,W30.Qe7,B30.Qg3,W31.Rf3,B31.Q

The merging of the dataframes was successful. There are a few remaining steps for data preparation. We need to make sure we have the correct datatypes in place and also remove rows with missing data or unnecessary data.

In [12]:
# remove rows with missing player ratings.
chess_data = chess_data[chess_data['B-ELO'] != 'None']
chess_data = chess_data[chess_data['W-ELO'] != 'None']

In [13]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359
0,2000.02.20,1/2-1/2,2851,2633,97,W1.e4,B1.e5,W2.Nf3,B2.Nc6,W3.Bb5,B3.a6,W4.Ba4,B4.Nf6,W5.O-O,B5.Be7,W6.Re1,B6.b5,W7.Bb3,B7.d6,W8.c3,B8.O-O,W9.h3,B9.Na5,W10.Bc2,B10.c5,W11.d4,B11.Qc7,W12.Nbd2,B12.Bd7,W13.Nf1,B13.cxd4,W14.cxd4,B14.Rac8,W15.Ne3,B15.Nc6,W16.d5,B16.Nb4,W17.Bb1,B17.a5,W18.a3,B18.Na6,W19.b4,B19.Ra8,W20.Bd2,B20.Rfc8,W21.Bd3,B21.Qb7,W22.g4,B22.g6,W23.Nf1,B23.axb4,W24.axb4,B24.Bd8,W25.Ng3,B25.Nc7,W26.Qe2,B26.Rxa1,W27.Rxa1,B27.Ra8,W28.Qe1,B28.Nfe8,W29.Qc1,B29.Ng7,W30.Rxa8,B30.Qxa8,W31.Bh6,B31.Nce8,W32.Qb2,B32.Qa4,W33.Kg2,B33.Bb6,W34.Bc2,B34.Qa7,W35.Bd3,B35.Qa4,W36.Ne2,B36.Nc7,W37.Nxe5,B37.dxe5,W38.Qxe5,B38.Nce8,W39.Bxg7,B39.Qd1,W40.Bh6,B40.Qxd3,W41.Qe7,B41.Ng7,W42.Ng3,B42.Qc2,W43.Qf6,B43.Nf5,W44.Qxb6,B44.Nh4+,W45.Kh2,B45.Nf3+,W46.Kg2,B46.Nh4+,W47.Kh2,B47.Nf3+,W48.Kg2,B48.Nh4+,W49.Kh2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000.02.12,1-0,2851,2555,147,W1.d4,B1.d5,W2.c4,B2.e6,W3.Nc3,B3.Nf6,W4.cxd5,B4.exd5,W5.Bg5,B5.c6,W6.Qc2,B6.Be7,W7.e3,B7.Nbd7,W8.Bd3,B8.O-O,W9.Nge2,B9.Re8,W10.O-O,B10.Nf8,W11.f3,B11.Ng6,W12.Rad1,B12.h6,W13.Bxf6,B13.Bxf6,W14.Bxg6,B14.fxg6,W15.e4,B15.g5,W16.e5,B16.Be7,W17.f4,B17.gxf4,W18.Nxf4,B18.Rf8,W19.Ng6,B19.Rxf1+,W20.Rxf1,B20.Be6,W21.Ne2,B21.Qd7,W22.h4,B22.Re8,W23.Ng3,B23.Bf7,W24.Nxe7+,B24.Rxe7,W25.Nf5,B25.Re6,W26.Nd6,B26.Bg6,W27.Qc3,B27.Rxd6,W28.exd6,B28.Qxd6,W29.Qa3,B29.Qb8,W30.Qe7,B30.Qg3,W31.Rf3,B31.Q

In [14]:
# We are interested in games that show wins or losses. We are not interested in draws. So we take out drawn games.
chess_data = chess_data[chess_data['Game Result'] != '1/2-1/2']

In [15]:
# remove rows where the number of game moves is 0.
chess_data = chess_data[chess_data['Num Moves'] != 0]

In [16]:
# remove rows where the number of game moves is over 75 for each player (75 * 2 players)
chess_data = chess_data[chess_data['Num Moves'] <= 150]

In [17]:
# we only want games with openings that we can analyze, so get all games that 
# have num moves at least 16 (8 move each side)
chess_data = chess_data[chess_data['Num Moves'] >= 16]

In [18]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359
1,2000.02.12,1-0,2851,2555,147,W1.d4,B1.d5,W2.c4,B2.e6,W3.Nc3,B3.Nf6,W4.cxd5,B4.exd5,W5.Bg5,B5.c6,W6.Qc2,B6.Be7,W7.e3,B7.Nbd7,W8.Bd3,B8.O-O,W9.Nge2,B9.Re8,W10.O-O,B10.Nf8,W11.f3,B11.Ng6,W12.Rad1,B12.h6,W13.Bxf6,B13.Bxf6,W14.Bxg6,B14.fxg6,W15.e4,B15.g5,W16.e5,B16.Be7,W17.f4,B17.gxf4,W18.Nxf4,B18.Rf8,W19.Ng6,B19.Rxf1+,W20.Rxf1,B20.Be6,W21.Ne2,B21.Qd7,W22.h4,B22.Re8,W23.Ng3,B23.Bf7,W24.Nxe7+,B24.Rxe7,W25.Nf5,B25.Re6,W26.Nd6,B26.Bg6,W27.Qc3,B27.Rxd6,W28.exd6,B28.Qxd6,W29.Qa3,B29.Qb8,W30.Qe7,B30.Qg3,W31.Rf3,B31.Qg4,W32.Qf8+,B32.Kh7,W33.Qf4,B33.Qd7,W34.Rg3,B34.Be4,W35.Qe5,B35.Qf7,W36.h5,B36.Qd7,W37.b4,B37.a6,W38.a4,B38.Kg8,W39.a5,B39.Kh7,W40.Kf2,B40.Kg8,W41.Qb8+,B41.Kh7,W42.Qf8,B42.Bc2,W43.Qf4,B43.Be4,W44.Qe5,B44.Kg8,W45.Ke3,B45.Kh7,W46.Ke2,B46.Kg8,W47.Qb8+,B47.Kh7,W48.Qf8,B48.Bc2,W49.Kd2,B49.Be4,W50.Kc1,B50.Qc7,W51.Rg4,B51.Qd7,W52.Rf4,B52.Bxg2,W53.Qf5+,B53.Qxf5,W54.Rxf5,B54.Kg8,W55.Kd2,B55.Bh3,W56.Rf4,B56.Be6,W57.Ke3,B57.Bf7,W58.Rf5,B58.Kf8,W59.Re5,B59.Be8,W60.Kf4,B60.Bf7,W61.Kg4,B61.Be8,W62.Kg3,B62.Bf7,W63.Kf4,B63.Be8,W64.Kf5,B64.Bxh5,W65.Ke6,B65.Bg6,W66.Re3,B66.Bc2,W67.Rf3+,B67.Kg8,W68.b5,B68.cxb5,W69.Kxd5,B69.b4,W70.Kc4,B70.b3,W71.Kc3,B71.g5,W72.d5,B72.g4,W73.Rf4,B73.h5,W74.d6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,2000.04.01,1-0,2851,2544,45,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.d6,W4.c3,B4.Nf6,W5.Qe2,B5.e5,W6.d4,B6.exd4,W7.cxd4,B7.a6,W8.e5,B8.Nd5,W9.exd6+,B9.Be6,W10.Bxc6+,B10.bxc6,W11.dxc5,B11.Qa5+,W12.Bd2,B12.Qxc5,W13.O-O,B13.Kd7,W14.Na3,B14.Bxd6,W15.Rab1,B15.Bf5,W16.Rbc1,B16.Rhe8,W17.Qd1,B17.Qa7,W18.Nc4,B18.Bc7,W19.Bc3,B19.f6,W20.Nd4,B20.Be6,W21.Nxc6,B21.Kxc6,W22.Bd4,B22.Qb8,

In [19]:
# we have too many columns because of all the NA values. Get dataframe up to col 155. That's 5 cols for misc data plus 150 cols
# for the chess moves (there will be a max of 150 moves)
chess_data = chess_data[chess_data.columns[:155]]

In [20]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
1,2000.02.12,1-0,2851,2555,147,W1.d4,B1.d5,W2.c4,B2.e6,W3.Nc3,B3.Nf6,W4.cxd5,B4.exd5,W5.Bg5,B5.c6,W6.Qc2,B6.Be7,W7.e3,B7.Nbd7,W8.Bd3,B8.O-O,W9.Nge2,B9.Re8,W10.O-O,B10.Nf8,W11.f3,B11.Ng6,W12.Rad1,B12.h6,W13.Bxf6,B13.Bxf6,W14.Bxg6,B14.fxg6,W15.e4,B15.g5,W16.e5,B16.Be7,W17.f4,B17.gxf4,W18.Nxf4,B18.Rf8,W19.Ng6,B19.Rxf1+,W20.Rxf1,B20.Be6,W21.Ne2,B21.Qd7,W22.h4,B22.Re8,W23.Ng3,B23.Bf7,W24.Nxe7+,B24.Rxe7,W25.Nf5,B25.Re6,W26.Nd6,B26.Bg6,W27.Qc3,B27.Rxd6,W28.exd6,B28.Qxd6,W29.Qa3,B29.Qb8,W30.Qe7,B30.Qg3,W31.Rf3,B31.Qg4,W32.Qf8+,B32.Kh7,W33.Qf4,B33.Qd7,W34.Rg3,B34.Be4,W35.Qe5,B35.Qf7,W36.h5,B36.Qd7,W37.b4,B37.a6,W38.a4,B38.Kg8,W39.a5,B39.Kh7,W40.Kf2,B40.Kg8,W41.Qb8+,B41.Kh7,W42.Qf8,B42.Bc2,W43.Qf4,B43.Be4,W44.Qe5,B44.Kg8,W45.Ke3,B45.Kh7,W46.Ke2,B46.Kg8,W47.Qb8+,B47.Kh7,W48.Qf8,B48.Bc2,W49.Kd2,B49.Be4,W50.Kc1,B50.Qc7,W51.Rg4,B51.Qd7,W52.Rf4,B52.Bxg2,W53.Qf5+,B53.Qxf5,W54.Rxf5,B54.Kg8,W55.Kd2,B55.Bh3,W56.Rf4,B56.Be6,W57.Ke3,B57.Bf7,W58.Rf5,B58.Kf8,W59.Re5,B59.Be8,W60.Kf4,B60.Bf7,W61.Kg4,B61.Be8,W62.Kg3,B62.Bf7,W63.Kf4,B63.Be8,W64.Kf5,B64.Bxh5,W65.Ke6,B65.Bg6,W66.Re3,B66.Bc2,W67.Rf3+,B67.Kg8,W68.b5,B68.cxb5,W69.Kxd5,B69.b4,W70.Kc4,B70.b3,W71.Kc3,B71.g5,W72.d5,B72.g4,W73.Rf4,B73.h5,W74.d6,<NA>,<NA>,<NA>
6,2000.04.01,1-0,2851,2544,45,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.d6,W4.c3,B4.Nf6,W5.Qe2,B5.e5,W6.d4,B6.exd4,W7.cxd4,B7.a6,W8.e5,B8.Nd5,W9.exd6+,B9.Be6,W10.Bxc6+,B10.bxc6,W11.dxc5,B11.Qa5+,W12.Bd2,B12.Qxc5,W13.O-O,B13.Kd7,W14.Na3,B14.Bxd6,W15.Rab1,B15.Bf5,W16.Rbc1,B16.Rhe8,W17.Qd1,B17.Qa7,W18.Nc4,B18.Bc7,W19.Bc3,B19.f6,W20.Nd4,B20.Be6,W21.Nxc6,B21.Kxc6,W22.Bd4,B22.Qb8,W23.Qa4+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,2000.01.20,1-0,2851,2748,90,W1.d4,B1.d5,W2.c4,B2.c6,W3.Nc3,B3.Nf6,W4.Nf3,B4.dxc4,W5.a4,B5.Bf5,W6.Ne5,B6.Nbd7,W7.Nxc4,B7.Qc7,W8.g3,B8.e5,W9.dxe5,B9.Nxe5,W10.Bf4,B10.Nfd7,W11.Bg2,B11.g5,W12.Ne3,B12.gxf4,W13.Nxf5,B13.O-O-O,W14.Qc2,B14.Ng4,W15.a5,B15.fxg3,W16.hxg3,B16.a6,W17.Ra4,B17.Ndf6,W18.Ne4,B18.Nxe4,W19.Bxe4,B19.h5,W20.Kf1,B20.Kb8,W21.Kg2,B21.Be7,W22.Nxe7,B22.Qxe7,W23.Bf3,B23.Ne5,W24.Bxh5,B24.Qe6,W25.Qc3,B25.f6,W26.Rah4,B26.Qf5,W27.Bf3,B27.Rxh4,W28.Rxh4,B28.Qb1,W29.Rh1,B29.Rd1,W30.Rxd1,B30.Qxd1,W31.b4,B31.Kc7,W32.Qc5,B32.Qd6,W33.Qxd6+,B33.Kxd6,W34.Be4,B34.Nc4,W35.Bd3,B35.Nb2,W36.f4,B36.Nd1,W37.g4,B37.Ne3+,W38.Kf3,B38.Nd5,W39.Ke4,B39.Nxb4,W40.Bc4,B40.c5,W41.g5,B41.fxg5,W42.fxg5,B42.Ke7,W43.Kf5,B43.Nc2,W44.Ke5,B44.Ne3,W45.Be6,B45.c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,2000.05.21,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh

In [21]:
# we only care about the year that the game took place. reformat the date col to reflect that
# try a string function on the df.
new_column = chess_data['Date'].str.slice(0, 4, 1)
chess_data['Date'] = new_column

In [22]:
chess_data.shape

(156418, 155)

In [23]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
1,2000,1-0,2851,2555,147,W1.d4,B1.d5,W2.c4,B2.e6,W3.Nc3,B3.Nf6,W4.cxd5,B4.exd5,W5.Bg5,B5.c6,W6.Qc2,B6.Be7,W7.e3,B7.Nbd7,W8.Bd3,B8.O-O,W9.Nge2,B9.Re8,W10.O-O,B10.Nf8,W11.f3,B11.Ng6,W12.Rad1,B12.h6,W13.Bxf6,B13.Bxf6,W14.Bxg6,B14.fxg6,W15.e4,B15.g5,W16.e5,B16.Be7,W17.f4,B17.gxf4,W18.Nxf4,B18.Rf8,W19.Ng6,B19.Rxf1+,W20.Rxf1,B20.Be6,W21.Ne2,B21.Qd7,W22.h4,B22.Re8,W23.Ng3,B23.Bf7,W24.Nxe7+,B24.Rxe7,W25.Nf5,B25.Re6,W26.Nd6,B26.Bg6,W27.Qc3,B27.Rxd6,W28.exd6,B28.Qxd6,W29.Qa3,B29.Qb8,W30.Qe7,B30.Qg3,W31.Rf3,B31.Qg4,W32.Qf8+,B32.Kh7,W33.Qf4,B33.Qd7,W34.Rg3,B34.Be4,W35.Qe5,B35.Qf7,W36.h5,B36.Qd7,W37.b4,B37.a6,W38.a4,B38.Kg8,W39.a5,B39.Kh7,W40.Kf2,B40.Kg8,W41.Qb8+,B41.Kh7,W42.Qf8,B42.Bc2,W43.Qf4,B43.Be4,W44.Qe5,B44.Kg8,W45.Ke3,B45.Kh7,W46.Ke2,B46.Kg8,W47.Qb8+,B47.Kh7,W48.Qf8,B48.Bc2,W49.Kd2,B49.Be4,W50.Kc1,B50.Qc7,W51.Rg4,B51.Qd7,W52.Rf4,B52.Bxg2,W53.Qf5+,B53.Qxf5,W54.Rxf5,B54.Kg8,W55.Kd2,B55.Bh3,W56.Rf4,B56.Be6,W57.Ke3,B57.Bf7,W58.Rf5,B58.Kf8,W59.Re5,B59.Be8,W60.Kf4,B60.Bf7,W61.Kg4,B61.Be8,W62.Kg3,B62.Bf7,W63.Kf4,B63.Be8,W64.Kf5,B64.Bxh5,W65.Ke6,B65.Bg6,W66.Re3,B66.Bc2,W67.Rf3+,B67.Kg8,W68.b5,B68.cxb5,W69.Kxd5,B69.b4,W70.Kc4,B70.b3,W71.Kc3,B71.g5,W72.d5,B72.g4,W73.Rf4,B73.h5,W74.d6,<NA>,<NA>,<NA>
6,2000,1-0,2851,2544,45,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.d6,W4.c3,B4.Nf6,W5.Qe2,B5.e5,W6.d4,B6.exd4,W7.cxd4,B7.a6,W8.e5,B8.Nd5,W9.exd6+,B9.Be6,W10.Bxc6+,B10.bxc6,W11.dxc5,B11.Qa5+,W12.Bd2,B12.Qxc5,W13.O-O,B13.Kd7,W14.Na3,B14.Bxd6,W15.Rab1,B15.Bf5,W16.Rbc1,B16.Rhe8,W17.Qd1,B17.Qa7,W18.Nc4,B18.Bc7,W19.Bc3,B19.f6,W20.Nd4,B20.Be6,W21.Nxc6,B21.Kxc6,W22.Bd4,B22.Qb8,W23.Qa4+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,2000,1-0,2851,2748,90,W1.d4,B1.d5,W2.c4,B2.c6,W3.Nc3,B3.Nf6,W4.Nf3,B4.dxc4,W5.a4,B5.Bf5,W6.Ne5,B6.Nbd7,W7.Nxc4,B7.Qc7,W8.g3,B8.e5,W9.dxe5,B9.Nxe5,W10.Bf4,B10.Nfd7,W11.Bg2,B11.g5,W12.Ne3,B12.gxf4,W13.Nxf5,B13.O-O-O,W14.Qc2,B14.Ng4,W15.a5,B15.fxg3,W16.hxg3,B16.a6,W17.Ra4,B17.Ndf6,W18.Ne4,B18.Nxe4,W19.Bxe4,B19.h5,W20.Kf1,B20.Kb8,W21.Kg2,B21.Be7,W22.Nxe7,B22.Qxe7,W23.Bf3,B23.Ne5,W24.Bxh5,B24.Qe6,W25.Qc3,B25.f6,W26.Rah4,B26.Qf5,W27.Bf3,B27.Rxh4,W28.Rxh4,B28.Qb1,W29.Rh1,B29.Rd1,W30.Rxd1,B30.Qxd1,W31.b4,B31.Kc7,W32.Qc5,B32.Qd6,W33.Qxd6+,B33.Kxd6,W34.Be4,B34.Nc4,W35.Bd3,B35.Nb2,W36.f4,B36.Nd1,W37.g4,B37.Ne3+,W38.Kf3,B38.Nd5,W39.Ke4,B39.Nxb4,W40.Bc4,B40.c5,W41.g5,B41.fxg5,W42.fxg5,B42.Ke7,W43.Kf5,B43.Nc2,W44.Ke5,B44.Ne3,W45.Be6,B45.c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,2000,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3

In [24]:
# take out games without dates.
chess_data = chess_data[chess_data['Date'].str.isnumeric()]

In [25]:
# take out any that show * for Game Result
chess_data = chess_data[chess_data['Game Result'] != '*']

In [26]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
1,2000,1-0,2851,2555,147,W1.d4,B1.d5,W2.c4,B2.e6,W3.Nc3,B3.Nf6,W4.cxd5,B4.exd5,W5.Bg5,B5.c6,W6.Qc2,B6.Be7,W7.e3,B7.Nbd7,W8.Bd3,B8.O-O,W9.Nge2,B9.Re8,W10.O-O,B10.Nf8,W11.f3,B11.Ng6,W12.Rad1,B12.h6,W13.Bxf6,B13.Bxf6,W14.Bxg6,B14.fxg6,W15.e4,B15.g5,W16.e5,B16.Be7,W17.f4,B17.gxf4,W18.Nxf4,B18.Rf8,W19.Ng6,B19.Rxf1+,W20.Rxf1,B20.Be6,W21.Ne2,B21.Qd7,W22.h4,B22.Re8,W23.Ng3,B23.Bf7,W24.Nxe7+,B24.Rxe7,W25.Nf5,B25.Re6,W26.Nd6,B26.Bg6,W27.Qc3,B27.Rxd6,W28.exd6,B28.Qxd6,W29.Qa3,B29.Qb8,W30.Qe7,B30.Qg3,W31.Rf3,B31.Qg4,W32.Qf8+,B32.Kh7,W33.Qf4,B33.Qd7,W34.Rg3,B34.Be4,W35.Qe5,B35.Qf7,W36.h5,B36.Qd7,W37.b4,B37.a6,W38.a4,B38.Kg8,W39.a5,B39.Kh7,W40.Kf2,B40.Kg8,W41.Qb8+,B41.Kh7,W42.Qf8,B42.Bc2,W43.Qf4,B43.Be4,W44.Qe5,B44.Kg8,W45.Ke3,B45.Kh7,W46.Ke2,B46.Kg8,W47.Qb8+,B47.Kh7,W48.Qf8,B48.Bc2,W49.Kd2,B49.Be4,W50.Kc1,B50.Qc7,W51.Rg4,B51.Qd7,W52.Rf4,B52.Bxg2,W53.Qf5+,B53.Qxf5,W54.Rxf5,B54.Kg8,W55.Kd2,B55.Bh3,W56.Rf4,B56.Be6,W57.Ke3,B57.Bf7,W58.Rf5,B58.Kf8,W59.Re5,B59.Be8,W60.Kf4,B60.Bf7,W61.Kg4,B61.Be8,W62.Kg3,B62.Bf7,W63.Kf4,B63.Be8,W64.Kf5,B64.Bxh5,W65.Ke6,B65.Bg6,W66.Re3,B66.Bc2,W67.Rf3+,B67.Kg8,W68.b5,B68.cxb5,W69.Kxd5,B69.b4,W70.Kc4,B70.b3,W71.Kc3,B71.g5,W72.d5,B72.g4,W73.Rf4,B73.h5,W74.d6,<NA>,<NA>,<NA>
6,2000,1-0,2851,2544,45,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.d6,W4.c3,B4.Nf6,W5.Qe2,B5.e5,W6.d4,B6.exd4,W7.cxd4,B7.a6,W8.e5,B8.Nd5,W9.exd6+,B9.Be6,W10.Bxc6+,B10.bxc6,W11.dxc5,B11.Qa5+,W12.Bd2,B12.Qxc5,W13.O-O,B13.Kd7,W14.Na3,B14.Bxd6,W15.Rab1,B15.Bf5,W16.Rbc1,B16.Rhe8,W17.Qd1,B17.Qa7,W18.Nc4,B18.Bc7,W19.Bc3,B19.f6,W20.Nd4,B20.Be6,W21.Nxc6,B21.Kxc6,W22.Bd4,B22.Qb8,W23.Qa4+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,2000,1-0,2851,2748,90,W1.d4,B1.d5,W2.c4,B2.c6,W3.Nc3,B3.Nf6,W4.Nf3,B4.dxc4,W5.a4,B5.Bf5,W6.Ne5,B6.Nbd7,W7.Nxc4,B7.Qc7,W8.g3,B8.e5,W9.dxe5,B9.Nxe5,W10.Bf4,B10.Nfd7,W11.Bg2,B11.g5,W12.Ne3,B12.gxf4,W13.Nxf5,B13.O-O-O,W14.Qc2,B14.Ng4,W15.a5,B15.fxg3,W16.hxg3,B16.a6,W17.Ra4,B17.Ndf6,W18.Ne4,B18.Nxe4,W19.Bxe4,B19.h5,W20.Kf1,B20.Kb8,W21.Kg2,B21.Be7,W22.Nxe7,B22.Qxe7,W23.Bf3,B23.Ne5,W24.Bxh5,B24.Qe6,W25.Qc3,B25.f6,W26.Rah4,B26.Qf5,W27.Bf3,B27.Rxh4,W28.Rxh4,B28.Qb1,W29.Rh1,B29.Rd1,W30.Rxd1,B30.Qxd1,W31.b4,B31.Kc7,W32.Qc5,B32.Qd6,W33.Qxd6+,B33.Kxd6,W34.Be4,B34.Nc4,W35.Bd3,B35.Nb2,W36.f4,B36.Nd1,W37.g4,B37.Ne3+,W38.Kf3,B38.Nd5,W39.Ke4,B39.Nxb4,W40.Bc4,B40.c5,W41.g5,B41.fxg5,W42.fxg5,B42.Ke7,W43.Kf5,B43.Nc2,W44.Ke5,B44.Ne3,W45.Be6,B45.c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,2000,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3

In [27]:
# we want the date to be a number.
chess_data = chess_data.astype({'Date': 'int'}, copy=True, errors='raise')

In [28]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
1,2000,1-0,2851,2555,147,W1.d4,B1.d5,W2.c4,B2.e6,W3.Nc3,B3.Nf6,W4.cxd5,B4.exd5,W5.Bg5,B5.c6,W6.Qc2,B6.Be7,W7.e3,B7.Nbd7,W8.Bd3,B8.O-O,W9.Nge2,B9.Re8,W10.O-O,B10.Nf8,W11.f3,B11.Ng6,W12.Rad1,B12.h6,W13.Bxf6,B13.Bxf6,W14.Bxg6,B14.fxg6,W15.e4,B15.g5,W16.e5,B16.Be7,W17.f4,B17.gxf4,W18.Nxf4,B18.Rf8,W19.Ng6,B19.Rxf1+,W20.Rxf1,B20.Be6,W21.Ne2,B21.Qd7,W22.h4,B22.Re8,W23.Ng3,B23.Bf7,W24.Nxe7+,B24.Rxe7,W25.Nf5,B25.Re6,W26.Nd6,B26.Bg6,W27.Qc3,B27.Rxd6,W28.exd6,B28.Qxd6,W29.Qa3,B29.Qb8,W30.Qe7,B30.Qg3,W31.Rf3,B31.Qg4,W32.Qf8+,B32.Kh7,W33.Qf4,B33.Qd7,W34.Rg3,B34.Be4,W35.Qe5,B35.Qf7,W36.h5,B36.Qd7,W37.b4,B37.a6,W38.a4,B38.Kg8,W39.a5,B39.Kh7,W40.Kf2,B40.Kg8,W41.Qb8+,B41.Kh7,W42.Qf8,B42.Bc2,W43.Qf4,B43.Be4,W44.Qe5,B44.Kg8,W45.Ke3,B45.Kh7,W46.Ke2,B46.Kg8,W47.Qb8+,B47.Kh7,W48.Qf8,B48.Bc2,W49.Kd2,B49.Be4,W50.Kc1,B50.Qc7,W51.Rg4,B51.Qd7,W52.Rf4,B52.Bxg2,W53.Qf5+,B53.Qxf5,W54.Rxf5,B54.Kg8,W55.Kd2,B55.Bh3,W56.Rf4,B56.Be6,W57.Ke3,B57.Bf7,W58.Rf5,B58.Kf8,W59.Re5,B59.Be8,W60.Kf4,B60.Bf7,W61.Kg4,B61.Be8,W62.Kg3,B62.Bf7,W63.Kf4,B63.Be8,W64.Kf5,B64.Bxh5,W65.Ke6,B65.Bg6,W66.Re3,B66.Bc2,W67.Rf3+,B67.Kg8,W68.b5,B68.cxb5,W69.Kxd5,B69.b4,W70.Kc4,B70.b3,W71.Kc3,B71.g5,W72.d5,B72.g4,W73.Rf4,B73.h5,W74.d6,<NA>,<NA>,<NA>
6,2000,1-0,2851,2544,45,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.d6,W4.c3,B4.Nf6,W5.Qe2,B5.e5,W6.d4,B6.exd4,W7.cxd4,B7.a6,W8.e5,B8.Nd5,W9.exd6+,B9.Be6,W10.Bxc6+,B10.bxc6,W11.dxc5,B11.Qa5+,W12.Bd2,B12.Qxc5,W13.O-O,B13.Kd7,W14.Na3,B14.Bxd6,W15.Rab1,B15.Bf5,W16.Rbc1,B16.Rhe8,W17.Qd1,B17.Qa7,W18.Nc4,B18.Bc7,W19.Bc3,B19.f6,W20.Nd4,B20.Be6,W21.Nxc6,B21.Kxc6,W22.Bd4,B22.Qb8,W23.Qa4+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,2000,1-0,2851,2748,90,W1.d4,B1.d5,W2.c4,B2.c6,W3.Nc3,B3.Nf6,W4.Nf3,B4.dxc4,W5.a4,B5.Bf5,W6.Ne5,B6.Nbd7,W7.Nxc4,B7.Qc7,W8.g3,B8.e5,W9.dxe5,B9.Nxe5,W10.Bf4,B10.Nfd7,W11.Bg2,B11.g5,W12.Ne3,B12.gxf4,W13.Nxf5,B13.O-O-O,W14.Qc2,B14.Ng4,W15.a5,B15.fxg3,W16.hxg3,B16.a6,W17.Ra4,B17.Ndf6,W18.Ne4,B18.Nxe4,W19.Bxe4,B19.h5,W20.Kf1,B20.Kb8,W21.Kg2,B21.Be7,W22.Nxe7,B22.Qxe7,W23.Bf3,B23.Ne5,W24.Bxh5,B24.Qe6,W25.Qc3,B25.f6,W26.Rah4,B26.Qf5,W27.Bf3,B27.Rxh4,W28.Rxh4,B28.Qb1,W29.Rh1,B29.Rd1,W30.Rxd1,B30.Qxd1,W31.b4,B31.Kc7,W32.Qc5,B32.Qd6,W33.Qxd6+,B33.Kxd6,W34.Be4,B34.Nc4,W35.Bd3,B35.Nb2,W36.f4,B36.Nd1,W37.g4,B37.Ne3+,W38.Kf3,B38.Nd5,W39.Ke4,B39.Nxb4,W40.Bc4,B40.c5,W41.g5,B41.fxg5,W42.fxg5,B42.Ke7,W43.Kf5,B43.Nc2,W44.Ke5,B44.Ne3,W45.Be6,B45.c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,2000,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3

In [29]:
chess_data.dtypes

Date            int32
Game Result    object
W-ELO          object
B-ELO          object
Num Moves       int64
                ...  
145            string
146            string
147            string
148            string
149            string
Length: 155, dtype: object

In [30]:
# convert Num Moves to 32 bit int to reduce df size
chess_data = chess_data.astype({'Num Moves': 'int'}, copy=True, errors='raise')

In [31]:
# the dtypes for B-ELO and W-ELO need to be the same. So we convert W-ELO and B-ELO to int
chess_data = chess_data.astype({'B-ELO': 'int'}, copy=True, errors='raise')
chess_data = chess_data.astype({'W-ELO': 'int'}, copy=True, errors='raise')

In [32]:
# change game result to string
chess_data = chess_data.astype({'Game Result': 'string'}, copy=True, errors='raise')

In [33]:
chess_data.dtypes

Date            int32
Game Result    string
W-ELO           int32
B-ELO           int32
Num Moves       int32
                ...  
145            string
146            string
147            string
148            string
149            string
Length: 155, dtype: object

In [34]:
# This code will be used to rename the col names.
def createMovesList1(x): # Number of moves is <= 9
  size = x.size
  return x.str.slice(3,size,1)
def createMovesList2(x): # Number of moves is <= 99
  size = x.size
  return x.str.slice(4,size,1)
def createMovesList3(x): # Number of moves is <= 100
  size = x.size
  return x.str.slice(5,size,1)

In [35]:
# using this code to rename the cols.
chess_data.iloc[:,5:23] = chess_data.iloc[:, 5:23].apply(createMovesList1)
chess_data.iloc[:,23:113] = chess_data.iloc[:, 23:113].apply(createMovesList2)
chess_data.iloc[:,113:] = chess_data.iloc[:, 113:].apply(createMovesList3)
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
1,2000,1-0,2851,2555,147,d4,d5,c4,e6,Nc3,Nf6,cxd5,exd5,Bg5,c6,Qc2,Be7,e3,Nbd7,Bd3,O-O,Nge2,Re8,O-O,Nf8,f3,Ng6,Rad1,h6,Bxf6,Bxf6,Bxg6,fxg6,e4,g5,e5,Be7,f4,gxf4,Nxf4,Rf8,Ng6,Rxf1+,Rxf1,Be6,Ne2,Qd7,h4,Re8,Ng3,Bf7,Nxe7+,Rxe7,Nf5,Re6,Nd6,Bg6,Qc3,Rxd6,exd6,Qxd6,Qa3,Qb8,Qe7,Qg3,Rf3,Qg4,Qf8+,Kh7,Qf4,Qd7,Rg3,Be4,Qe5,Qf7,h5,Qd7,b4,a6,a4,Kg8,a5,Kh7,Kf2,Kg8,Qb8+,Kh7,Qf8,Bc2,Qf4,Be4,Qe5,Kg8,Ke3,Kh7,Ke2,Kg8,Qb8+,Kh7,Qf8,Bc2,Kd2,Be4,Kc1,Qc7,Rg4,Qd7,Rf4,Bxg2,Qf5+,Qxf5,Rxf5,Kg8,d2,h3,f4,e6,e3,f7,f5,f8,e5,e8,f4,f7,g4,e8,g3,f7,f4,e8,f5,xh5,e6,g6,e3,c2,f3+,g8,5,xb5,xd5,4,c4,3,c3,5,5,4,f4,5,6,<NA>,<NA>,<NA>
6,2000,1-0,2851,2544,45,e4,c5,Nf3,Nc6,Bb5,d6,c3,Nf6,Qe2,e5,d4,exd4,cxd4,a6,e5,Nd5,exd6+,Be6,Bxc6+,bxc6,dxc5,Qa5+,Bd2,Qxc5,O-O,Kd7,Na3,Bxd6,Rab1,Bf5,Rbc1,Rhe8,Qd1,Qa7,Nc4,Bc7,Bc3,f6,Nd4,Be6,Nxc6,Kxc6,Bd4,Qb8,Qa4+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,2000,1-0,2851,2748,90,d4,d5,c4,c6,Nc3,Nf6,Nf3,dxc4,a4,Bf5,Ne5,Nbd7,Nxc4,Qc7,g3,e5,dxe5,Nxe5,Bf4,Nfd7,Bg2,g5,Ne3,gxf4,Nxf5,O-O-O,Qc2,Ng4,a5,fxg3,hxg3,a6,Ra4,Ndf6,Ne4,Nxe4,Bxe4,h5,Kf1,Kb8,Kg2,Be7,Nxe7,Qxe7,Bf3,Ne5,Bxh5,Qe6,Qc3,f6,Rah4,Qf5,Bf3,Rxh4,Rxh4,Qb1,Rh1,Rd1,Rxd1,Qxd1,b4,Kc7,Qc5,Qd6,Qxd6+,Kxd6,Be4,Nc4,Bd3,Nb2,f4,Nd1,g4,Ne3+,Kf3,Nd5,Ke4,Nxb4,Bc4,c5,g5,fxg5,fxg5,Ke7,Kf5,Nc2,Ke5,Ne3,Be6,c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,2000,1-0,2851,2751,69,e4,e6,d4,d5,Nc3,Nf6,Bg5,dxe4,Nxe4,Nbd7,Nf3,Be7,Nxf6+,Bxf6,h4,O-O,Bd3,c5,Qe2,cxd4,Qe4,g6,O-O-O,Qa5,Bxf6,Nxf6,Qxd4,Nh5,a3,Rd8,Qe3,Bd7,g4,Nf6,Qf4,Nd5,Qh6,Nf6,Ng5,Bc6,Bxg6,hxg6,Nxe6,fxe6,Qxg6+,Kh8,Qxf6+,Kh7,Rhe1,Rxd1+,Rxd1,Qc5,g5,Rf8,Qh6+,Kg8,Qxe6+,Kg7,Qh6+,Kg8,Qg6+,Kh8,Qh6+,Kg8,Qe6+,Kg7,Rd6,Be8,Qe7+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [36]:
# This function will be used to concatenate two strings from two arrays
def combineStrings(a1,a2):
  new_list = []
  for i in range(0, a1.size):
    new_list.append(a1[i] + a2[i])
  return new_list

In [37]:
# Creating a list of integer from 1 to 75
# Then covert list type from int to string
a = np.arange(1,76,1)
a_str = list(map(str,a))

In [38]:
# Create move piece names.
w_pieces = np.full(75, ['W'])
b_pieces = np.full(75, ['B'])
w_new_list = combineStrings(w_pieces, a_str)
b_new_list = combineStrings(b_pieces, a_str)

In [39]:
# Combine list but alternate between white and black piece.
values = [None] * (len(w_new_list) + len(b_new_list))
values[::2] = w_new_list
values[1::2] = b_new_list

In [40]:
# rename columns to show W1, B1, W2, B2, .....
keys = np.arange(0,150,1)
dictionary = dict(zip(keys,values))
chess_data.rename(columns = dictionary, inplace=True)
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
1,2000,1-0,2851,2555,147,d4,d5,c4,e6,Nc3,Nf6,cxd5,exd5,Bg5,c6,Qc2,Be7,e3,Nbd7,Bd3,O-O,Nge2,Re8,O-O,Nf8,f3,Ng6,Rad1,h6,Bxf6,Bxf6,Bxg6,fxg6,e4,g5,e5,Be7,f4,gxf4,Nxf4,Rf8,Ng6,Rxf1+,Rxf1,Be6,Ne2,Qd7,h4,Re8,Ng3,Bf7,Nxe7+,Rxe7,Nf5,Re6,Nd6,Bg6,Qc3,Rxd6,exd6,Qxd6,Qa3,Qb8,Qe7,Qg3,Rf3,Qg4,Qf8+,Kh7,Qf4,Qd7,Rg3,Be4,Qe5,Qf7,h5,Qd7,b4,a6,a4,Kg8,a5,Kh7,Kf2,Kg8,Qb8+,Kh7,Qf8,Bc2,Qf4,Be4,Qe5,Kg8,Ke3,Kh7,Ke2,Kg8,Qb8+,Kh7,Qf8,Bc2,Kd2,Be4,Kc1,Qc7,Rg4,Qd7,Rf4,Bxg2,Qf5+,Qxf5,Rxf5,Kg8,d2,h3,f4,e6,e3,f7,f5,f8,e5,e8,f4,f7,g4,e8,g3,f7,f4,e8,f5,xh5,e6,g6,e3,c2,f3+,g8,5,xb5,xd5,4,c4,3,c3,5,5,4,f4,5,6,<NA>,<NA>,<NA>
6,2000,1-0,2851,2544,45,e4,c5,Nf3,Nc6,Bb5,d6,c3,Nf6,Qe2,e5,d4,exd4,cxd4,a6,e5,Nd5,exd6+,Be6,Bxc6+,bxc6,dxc5,Qa5+,Bd2,Qxc5,O-O,Kd7,Na3,Bxd6,Rab1,Bf5,Rbc1,Rhe8,Qd1,Qa7,Nc4,Bc7,Bc3,f6,Nd4,Be6,Nxc6,Kxc6,Bd4,Qb8,Qa4+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,2000,1-0,2851,2748,90,d4,d5,c4,c6,Nc3,Nf6,Nf3,dxc4,a4,Bf5,Ne5,Nbd7,Nxc4,Qc7,g3,e5,dxe5,Nxe5,Bf4,Nfd7,Bg2,g5,Ne3,gxf4,Nxf5,O-O-O,Qc2,Ng4,a5,fxg3,hxg3,a6,Ra4,Ndf6,Ne4,Nxe4,Bxe4,h5,Kf1,Kb8,Kg2,Be7,Nxe7,Qxe7,Bf3,Ne5,Bxh5,Qe6,Qc3,f6,Rah4,Qf5,Bf3,Rxh4,Rxh4,Qb1,Rh1,Rd1,Rxd1,Qxd1,b4,Kc7,Qc5,Qd6,Qxd6+,Kxd6,Be4,Nc4,Bd3,Nb2,f4,Nd1,g4,Ne3+,Kf3,Nd5,Ke4,Nxb4,Bc4,c5,g5,fxg5,fxg5,Ke7,Kf5,Nc2,Ke5,Ne3,Be6,c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,2000,1-0,2851,2751,69,e4,e6,d4,d5,Nc3,Nf6,Bg5,dxe4,Nxe4,Nbd7,Nf3,Be7,Nxf6+,Bxf6,h4,O-O,Bd3,c5,Qe2,cxd4,Qe4,g6,O-O-O,Qa5,Bxf6,Nxf6,Qxd4,Nh5,a3,Rd8,Qe3,Bd7,g4,Nf6,Qf4,Nd5,Qh6,Nf6,Ng5,Bc6,Bxg6,hxg6,Nxe6,fxe6,Qxg6+,Kh8,Qxf6+,Kh7,Rhe1,Rxd1+,Rxd1,Qc5,g5,Rf8,Qh6+,Kg8,Qxe6+,Kg7,Qh6+,Kg8,Qg6+,Kh8,Qh6+,Kg8,Qe6+,Kg7,Rd6,Be8,Qe7+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,

In [41]:
total_games = chess_data.index.values.size + 1

def createGames(total_games):
  new_list = []
  for i in range(1, total_games):
    new_list.append('Game' + str(i))
  return new_list

index_list = createGames(total_games)
chess_data.index = index_list

In [42]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
Game1,2000,1-0,2851,2555,147,d4,d5,c4,e6,Nc3,Nf6,cxd5,exd5,Bg5,c6,Qc2,Be7,e3,Nbd7,Bd3,O-O,Nge2,Re8,O-O,Nf8,f3,Ng6,Rad1,h6,Bxf6,Bxf6,Bxg6,fxg6,e4,g5,e5,Be7,f4,gxf4,Nxf4,Rf8,Ng6,Rxf1+,Rxf1,Be6,Ne2,Qd7,h4,Re8,Ng3,Bf7,Nxe7+,Rxe7,Nf5,Re6,Nd6,Bg6,Qc3,Rxd6,exd6,Qxd6,Qa3,Qb8,Qe7,Qg3,Rf3,Qg4,Qf8+,Kh7,Qf4,Qd7,Rg3,Be4,Qe5,Qf7,h5,Qd7,b4,a6,a4,Kg8,a5,Kh7,Kf2,Kg8,Qb8+,Kh7,Qf8,Bc2,Qf4,Be4,Qe5,Kg8,Ke3,Kh7,Ke2,Kg8,Qb8+,Kh7,Qf8,Bc2,Kd2,Be4,Kc1,Qc7,Rg4,Qd7,Rf4,Bxg2,Qf5+,Qxf5,Rxf5,Kg8,d2,h3,f4,e6,e3,f7,f5,f8,e5,e8,f4,f7,g4,e8,g3,f7,f4,e8,f5,xh5,e6,g6,e3,c2,f3+,g8,5,xb5,xd5,4,c4,3,c3,5,5,4,f4,5,6,<NA>,<NA>,<NA>
Game2,2000,1-0,2851,2544,45,e4,c5,Nf3,Nc6,Bb5,d6,c3,Nf6,Qe2,e5,d4,exd4,cxd4,a6,e5,Nd5,exd6+,Be6,Bxc6+,bxc6,dxc5,Qa5+,Bd2,Qxc5,O-O,Kd7,Na3,Bxd6,Rab1,Bf5,Rbc1,Rhe8,Qd1,Qa7,Nc4,Bc7,Bc3,f6,Nd4,Be6,Nxc6,Kxc6,Bd4,Qb8,Qa4+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game3,2000,1-0,2851,2748,90,d4,d5,c4,c6,Nc3,Nf6,Nf3,dxc4,a4,Bf5,Ne5,Nbd7,Nxc4,Qc7,g3,e5,dxe5,Nxe5,Bf4,Nfd7,Bg2,g5,Ne3,gxf4,Nxf5,O-O-O,Qc2,Ng4,a5,fxg3,hxg3,a6,Ra4,Ndf6,Ne4,Nxe4,Bxe4,h5,Kf1,Kb8,Kg2,Be7,Nxe7,Qxe7,Bf3,Ne5,Bxh5,Qe6,Qc3,f6,Rah4,Qf5,Bf3,Rxh4,Rxh4,Qb1,Rh1,Rd1,Rxd1,Qxd1,b4,Kc7,Qc5,Qd6,Qxd6+,Kxd6,Be4,Nc4,Bd3,Nb2,f4,Nd1,g4,Ne3+,Kf3,Nd5,Ke4,Nxb4,Bc4,c5,g5,fxg5,fxg5,Ke7,Kf5,Nc2,Ke5,Ne3,Be6,c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game4,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game5,2000,1-0,2851,2751,69,e4,e6,d4,d5,Nc3,Nf6,Bg5,dxe4,Nxe4,Nbd7,Nf3,Be7,Nxf6+,Bxf6,h4,O-O,Bd3,c5,Qe2,cxd4,Qe4,g6,O-O-O,Qa5,Bxf6,Nxf6,Qxd4,Nh5,a3,Rd8,Qe3,Bd7,g4,Nf6,Qf4,Nd5,Qh6,Nf6,Ng5,Bc6,Bxg6,hxg6,Nxe6,fxe6,Qxg6+,Kh8,Qxf6+,Kh7,Rhe1,Rxd1+,Rxd1,Qc5,g5,Rf8,Qh6+,Kg8,Qxe6+,Kg7,Qh6+,Kg8,Qg6+,Kh8,Qh6+,Kg8,Qe6+,Kg7,Rd6,Be8,Qe7+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,

We finally have a dataset that has been prepared and formatted to our needs.

In [43]:
chess_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 156417 entries, Game1 to Game156417
Columns: 155 entries, Date to B75
dtypes: int32(4), string(151)
memory usage: 183.8+ MB


In [44]:
chess_data.head(100)

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
Game1,2000,1-0,2851,2555,147,d4,d5,c4,e6,Nc3,Nf6,cxd5,exd5,Bg5,c6,Qc2,Be7,e3,Nbd7,Bd3,O-O,Nge2,Re8,O-O,Nf8,f3,Ng6,Rad1,h6,Bxf6,Bxf6,Bxg6,fxg6,e4,g5,e5,Be7,f4,gxf4,Nxf4,Rf8,Ng6,Rxf1+,Rxf1,Be6,Ne2,Qd7,h4,Re8,Ng3,Bf7,Nxe7+,Rxe7,Nf5,Re6,Nd6,Bg6,Qc3,Rxd6,exd6,Qxd6,Qa3,Qb8,Qe7,Qg3,Rf3,Qg4,Qf8+,Kh7,Qf4,Qd7,Rg3,Be4,Qe5,Qf7,h5,Qd7,b4,a6,a4,Kg8,a5,Kh7,Kf2,Kg8,Qb8+,Kh7,Qf8,Bc2,Qf4,Be4,Qe5,Kg8,Ke3,Kh7,Ke2,Kg8,Qb8+,Kh7,Qf8,Bc2,Kd2,Be4,Kc1,Qc7,Rg4,Qd7,Rf4,Bxg2,Qf5+,Qxf5,Rxf5,Kg8,d2,h3,f4,e6,e3,f7,f5,f8,e5,e8,f4,f7,g4,e8,g3,f7,f4,e8,f5,xh5,e6,g6,e3,c2,f3+,g8,5,xb5,xd5,4,c4,3,c3,5,5,4,f4,5,6,<NA>,<NA>,<NA>
Game2,2000,1-0,2851,2544,45,e4,c5,Nf3,Nc6,Bb5,d6,c3,Nf6,Qe2,e5,d4,exd4,cxd4,a6,e5,Nd5,exd6+,Be6,Bxc6+,bxc6,dxc5,Qa5+,Bd2,Qxc5,O-O,Kd7,Na3,Bxd6,Rab1,Bf5,Rbc1,Rhe8,Qd1,Qa7,Nc4,Bc7,Bc3,f6,Nd4,Be6,Nxc6,Kxc6,Bd4,Qb8,Qa4+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game3,2000,1-0,2851,2748,90,d4,d5,c4,c6,Nc3,Nf6,Nf3,dxc4,a4,Bf5,Ne5,Nbd7,Nxc4,Qc7,g3,e5,dxe5,Nxe5,Bf4,Nfd7,Bg2,g5,Ne3,gxf4,Nxf5,O-O-O,Qc2,Ng4,a5,fxg3,hxg3,a6,Ra4,Ndf6,Ne4,Nxe4,Bxe4,h5,Kf1,Kb8,Kg2,Be7,Nxe7,Qxe7,Bf3,Ne5,Bxh5,Qe6,Qc3,f6,Rah4,Qf5,Bf3,Rxh4,Rxh4,Qb1,Rh1,Rd1,Rxd1,Qxd1,b4,Kc7,Qc5,Qd6,Qxd6+,Kxd6,Be4,Nc4,Bd3,Nb2,f4,Nd1,g4,Ne3+,Kf3,Nd5,Ke4,Nxb4,Bc4,c5,g5,fxg5,fxg5,Ke7,Kf5,Nc2,Ke5,Ne3,Be6,c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game4,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game5,2000,1-0,2851,2751,69,e4,e6,d4,d5,Nc3,Nf6,Bg5,dxe4,Nxe4,Nbd7,Nf3,Be7,Nxf6+,Bxf6,h4,O-O,Bd3,c5,Qe2,cxd4,Qe4,g6,O-O-O,Qa5,Bxf6,Nxf6,Qxd4,Nh5,a3,Rd8,Qe3,Bd7,g4,Nf6,Qf4,Nd5,Qh6,Nf6,Ng5,Bc6,Bxg6,hxg6,Nxe6,fxe6,Qxg6+,Kh8,Qxf6+,Kh7,Rhe1,Rxd1+,Rxd1,Qc5,g5,Rf8,Qh6+,Kg8,Qxe6+,Kg7,Qh6+,Kg8,Qg6+,Kh8,Qh6+,Kg8,Qe6+,Kg7,Rd6,Be8,Qe7+,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,

In [45]:
# we will upload pkl file to github and use it for visualization
chess_data.to_pickle("./chess_data.pkl", compression = 'zip')